In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time

In [116]:
col_dict = {"corp_id": str, "corp_name": str, "website": str}
corp = pd.read_excel('corp_website.xlsx', dtype=col_dict)

In [117]:
corp_id_list = list(corp["corp_id"])

In [118]:
corp

,corp_id,corp_name,corp_name_web,corp_name_web_eng,website,check
0,22099131,台灣積體電路製造股份有限公司,NaN,NaN,NaN,NaN
1,01615407,全翔食品行,NaN,NaN,NaN,NaN
2,7788,NaN,NaN,NaN,NaN,NaN
3,12336458,NaN,NaN,NaN,NaN,NaN
4,8046542,NaN,NaN,NaN,NaN,NaN
5,84149961,NaN,NaN,NaN,NaN,NaN


In [119]:
corp_id_list

['22099131', '01615407', '7788', '12336458', '8046542', '84149961']

In [100]:
#selenium settings
options = Options()
options.add_argument("--disable-gpu");
chrome = webdriver.Chrome('./chromedriver',options=options)
#設定等待時間
chrome.implicitly_wait(5)
chrome.set_page_load_timeout(30)

In [120]:
#loop
for id in corp_id_list:
    chrome.get("https://www.twincn.com/item.aspx?no="+str(id))
    try:
        table_tr_list = chrome.find_element("class name", "table-responsive").find_elements("tag name", "tr")
        table_list = []
        for tr in table_tr_list:
            #將每一個tr的資料根據td查詢出來，返回結果為list物件
            table_td_list = tr.find_elements("tag name", "td")
            row_list = []
            for td in table_td_list:    #遍歷每一個td
                row_list.append(td.text)    #取出表格的資料，並放入行列表裡
            table_list.append(row_list)
        for i in table_list:
            if i[0]=='網址':
                corp.loc[corp["corp_id"]==id,"website"] = i[1]
            elif i[0]=='公司名稱':
                corp.loc[corp["corp_id"]==id,"corp_name_web"] = i[1]
            elif i[0]=='英文名稱':
                corp.loc[corp["corp_id"]==id,"corp_name_web_eng"] = i[1]      
            corp.loc[corp["corp_id"]==id,"check"] = "Y"
    except:
        corp.loc[corp["corp_id"]==id,"check"] = "N"

In [121]:
corp

,corp_id,corp_name,corp_name_web,corp_name_web_eng,website,check
0,22099131,台灣積體電路製造股份有限公司,台灣積體電路製造股份有限公司,Taiwan Semiconductor Manufacturing Company Lim...,https://www.tsmc.com,Y
1,01615407,全翔食品行,全翔食品行,NaN,NaN,Y
2,7788,NaN,NaN,NaN,NaN,N
3,12336458,NaN,NaN,NaN,NaN,N
4,8046542,NaN,NaN,NaN,NaN,N
5,84149961,NaN,聯發科技股份有限公司,MEDIATEK INC.,http://www.mediatek.com,Y


In [122]:
corp.to_excel('corp_website_done.xlsx',index=False)